In [1852]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [1853]:
import pandas as pd

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
shark_df = pd.read_excel(url)

In [1854]:
shark_df = shark_df.drop(['pdf', 'href formula','href','Case Number','Case Number.1','original order','Unnamed: 21','Unnamed: 22','Unnamed: 11','Source'], axis=1)

In [1855]:
shark_df = shark_df.dropna(how='all')

In [1856]:
shark_df = shark_df.dropna(subset=['Country', 'State', 'Location'])

In [1857]:
shark_df.columns = shark_df.columns.str.lower()

In [1858]:
shark_df.columns

Index(['date', 'year', 'type', 'country', 'state', 'location', 'activity',
       'name', 'sex', 'age', 'injury', 'time', 'species '],
      dtype='object')

In [1859]:
usa_states = shark_df.loc[shark_df['country'] == 'USA', 'state']
usa_states_unique = usa_states.unique()
usa_states_unique

array(['Hawaii', 'Florida', 'California', 'South Carolina',
       'North Carolina', 'New York', 'New Jersey', 'Samoa', 'Louisiana',
       'Mississippi', 'Noirth Carolina', 'Georgia', 'Alabama', 'Texas',
       'Maryland', 'Maui', 'Oregon', 'Franklin County, Florida',
       'Virgin Islands', 'Maine', 'Bahamas', 'Delaware', 'Guam',
       'Cayman Islands', 'Rhode Island', 'Massachusetts', 'Washington',
       'Puerto Rico', 'Virginia', 'US Virgin Islands', 'Kentucky',
       'New Mexico', 'South Carolina ', 'Alaska', 'Missouri',
       'North Carolina ', 'Florida ', 'Connecticut', 'Pennsylvania',
       'Illinois', 'Wake Island', ' North Carolina', ' New Jersey',
       'New York ', 'CUBA'], dtype=object)

In [1860]:
#Remove cuba

row_index_cuba = shark_df.index[shark_df['state'].str.lower() == 'cuba'].tolist()

# Update 'country' and 'state' in the identified row
shark_df.loc[row_index_cuba, 'country'] = 'CUBA'
shark_df.loc[row_index_cuba, 'state'] = ''

# Standardize the state names in USA
shark_df['state'] = shark_df['state'].str.strip().str.lower().replace({
    'noirth carolina': 'north carolina',
    'maui': 'hawaii',
    'franklin county, florida': 'florida',
    'cuba': ''
})

# Check the unique values in 'state' for USA after replacement
usa_states = shark_df.loc[shark_df['country'] == 'USA', 'state']
usa_states_unique_after = usa_states.unique()
usa_states_unique_after

array(['hawaii', 'florida', 'california', 'south carolina',
       'north carolina', 'new york', 'new jersey', 'samoa', 'louisiana',
       'mississippi', 'georgia', 'alabama', 'texas', 'maryland', 'oregon',
       'virgin islands', 'maine', 'bahamas', 'delaware', 'guam',
       'cayman islands', 'rhode island', 'massachusetts', 'washington',
       'puerto rico', 'virginia', 'us virgin islands', 'kentucky',
       'new mexico', 'alaska', 'missouri', 'connecticut', 'pennsylvania',
       'illinois', 'wake island'], dtype=object)

In [1861]:
#Remove cuba

row_index_cuba = shark_df.index[shark_df['state'].str.lower() == 'cuba'].tolist()

# Update 'country' and 'state' in the identified row
shark_df.loc[row_index_cuba, 'country'] = 'CUBA'
shark_df.loc[row_index_cuba, 'state'] = ''

# Standardize the state names in USA
shark_df['state'] = shark_df['state'].str.strip().str.lower().replace({
    'noirth carolina': 'north carolina',
    'maui': 'hawaii',
    'franklin county, florida': 'florida',
    'cuba': ''
})

# Check the unique values in 'state' for USA after replacement
usa_states = shark_df.loc[shark_df['country'] == 'USA', 'state']
usa_states_unique_after = usa_states.unique()
usa_states_unique_after

array(['hawaii', 'florida', 'california', 'south carolina',
       'north carolina', 'new york', 'new jersey', 'samoa', 'louisiana',
       'mississippi', 'georgia', 'alabama', 'texas', 'maryland', 'oregon',
       'virgin islands', 'maine', 'bahamas', 'delaware', 'guam',
       'cayman islands', 'rhode island', 'massachusetts', 'washington',
       'puerto rico', 'virginia', 'us virgin islands', 'kentucky',
       'new mexico', 'alaska', 'missouri', 'connecticut', 'pennsylvania',
       'illinois', 'wake island'], dtype=object)

In [1862]:
# Nimrod: unified categories for Activity & Type

# grouping activity into upper-group (a less granual data)
import re

def clean_activity(activity):
    activity = str(activity).strip()

    if re.search(r'surf|cano|kayak|board|wind|paddle|padd|wing|kite|ski|sup', activity, flags=re.IGNORECASE):
        return 'water_sport'
    elif re.search(r'swim|play|snork|scuba|div|jump|squat|bath|Underwater|diving|float', activity, flags=re.IGNORECASE):
        return 'swimming'
    elif re.search(r'fish|scallop|lobster|wading|wade|prawn|spear', activity, flags=re.IGNORECASE):
        return 'fishing'
    elif re.search(r'torpedoed|sunk|troop|battle|disaster|squad|sank|fire|soldier|crash|plane|explod|capsized|yacht|fell|sinking|broke|wreck|USS'
                   , activity, flags=re.IGNORECASE):
        return 'disaster_war_accident'
    elif re.search(r'shark', activity, flags=re.IGNORECASE):
        return 'shark_related'
    else:
        return 'others'

# grouping type into upper-group (a less granual data)
def clean_type(type_):

    if pd.isna(type_):
        return 'Invalid'


    type_ = str(type_).strip()

    if re.search(r'Watercraft|disaster', type_, flags=re.IGNORECASE):
        return 'Unprovoked'
    elif re.search(r'Questionable', type_, flags=re.IGNORECASE):
        return 'Provoked'
    elif re.search(r'Unverified|Unconfirmed|\?|Boat|Under', type_, flags=re.IGNORECASE):
        return 'Invalid'
    else:
        return type_



In [1863]:
## performing cleaning using functions from above cell to create activity_grouped and upd
shark_df['activity_grouped'] = shark_df['activity'].apply(clean_activity)
shark_df['type'] = shark_df['type'].apply(clean_type)

In [1864]:
# clean 'sex' column to M, F and UNDEFINED
shark_df.sex = shark_df.sex.fillna('Undefined')
shark_df.sex = shark_df.sex.apply(lambda sex:sex.strip().upper() if isinstance(sex,str) else sex)
shark_df.sex = shark_df.sex.replace(['LLI'],['UNDEFINED'])

# clean 'name' column to fill nan as 'Unknown'
shark_df.name = shark_df.name.fillna('Unknown')

In [1865]:
shark_df_us = shark_df[shark_df['country']=='USA']
shark_df_us

,date,year,type,country,state,location,activity,name,sex,age,injury,time,species,activity_grouped
2,30 Dec-2023,2023.0,Unprovoked,USA,hawaii,"Baby Beach, Maui",Surfing,Jason Carter,M,39,FATAL,11h12,NaN,water_sport
16,05 Nov-2023,2023.0,Unprovoked,USA,florida,"Juno Beach, Palm Beach County",Swimming,Steven Reinhardt,M,66,Lacerations to right forearm,10h30,NaN,swimming
21,25 Oct 2023,2023.0,Unprovoked,USA,hawaii,"Pua’ena Point, Haleiwa, Oahu",Surfing,male,M,30,Bite to right thigh,15h05,8' tiger shark,water_sport
25,15 Oct 2023,2023.0,Unprovoked,USA,hawaii,"Hanalei Bay, Kauai",Surfing,Kevin Kanehe,M,50,Left leg and hand injured,16h00,10' tiger shark,water_sport
26,13 Oct-2023,2023.0,Unprovoked,USA,california,"Linda Mar, Pacific State Beach, San Mateo County",Surfing,male,M,52,Minor linjury to left lower leg,15h45,NaN,water_sport
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6879,Before 1958,0.0,Unprovoked,USA,florida,"Palm Beach, Palm Beach County",Standing,Horton Chase,M,NaN,Abrasions & bruises hip to ankle,NaN,NaN,others
6913,Before 1921,0.0,Unprovoked,USA,florida,"Gadsden Point, Tampa Bay",Fishing,James Kelley,M,NaN,2-inch lacerations,NaN,NaN,fishing
6918,Before 17-Jul-1916,0.0,Unprovoked,USA,north carolina,Somewhere between Hatteras and Beaufort,Swimming,"""youthful male""",M,NaN,"""Lost leg""",NaN,NaN,swimming
6922,Circa 1862,0.0,Unprovoked,USA,hawaii,Puna,NaN,"A ""chiefess""",F,NaN,Ankle bitten,NaN,NaN,others


In [1866]:
shark_df_us.to_csv('https://github.com/goldenrod-asters/Week-2-Quest/blob/main/shark_df_us.csv', index=False)

In [1867]:
shark_df_us

,date,year,type,country,state,location,activity,name,sex,age,injury,time,species,activity_grouped
2,30 Dec-2023,2023.0,Unprovoked,USA,hawaii,"Baby Beach, Maui",Surfing,Jason Carter,M,39,FATAL,11h12,NaN,water_sport
16,05 Nov-2023,2023.0,Unprovoked,USA,florida,"Juno Beach, Palm Beach County",Swimming,Steven Reinhardt,M,66,Lacerations to right forearm,10h30,NaN,swimming
21,25 Oct 2023,2023.0,Unprovoked,USA,hawaii,"Pua’ena Point, Haleiwa, Oahu",Surfing,male,M,30,Bite to right thigh,15h05,8' tiger shark,water_sport
25,15 Oct 2023,2023.0,Unprovoked,USA,hawaii,"Hanalei Bay, Kauai",Surfing,Kevin Kanehe,M,50,Left leg and hand injured,16h00,10' tiger shark,water_sport
26,13 Oct-2023,2023.0,Unprovoked,USA,california,"Linda Mar, Pacific State Beach, San Mateo County",Surfing,male,M,52,Minor linjury to left lower leg,15h45,NaN,water_sport
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6879,Before 1958,0.0,Unprovoked,USA,florida,"Palm Beach, Palm Beach County",Standing,Horton Chase,M,NaN,Abrasions & bruises hip to ankle,NaN,NaN,others
6913,Before 1921,0.0,Unprovoked,USA,florida,"Gadsden Point, Tampa Bay",Fishing,James Kelley,M,NaN,2-inch lacerations,NaN,NaN,fishing
6918,Before 17-Jul-1916,0.0,Unprovoked,USA,north carolina,Somewhere between Hatteras and Beaufort,Swimming,"""youthful male""",M,NaN,"""Lost leg""",NaN,NaN,swimming
6922,Circa 1862,0.0,Unprovoked,USA,hawaii,Puna,NaN,"A ""chiefess""",F,NaN,Ankle bitten,NaN,NaN,others


# 1. Clean cloumn Year

In [1868]:
# Check how many empty cells are in column year
shark_df_us['year'].isna().sum()

0

In [1869]:
# Only one empty cell in cloumn year, so we can replace with the Mode of column year
shark_df_us.year.fillna(shark_df_us.year.mode()[0], inplace=True)

C:\Users\vonke\AppData\Local\Temp\ipykernel_15648\2518628143.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us.year.fillna(shark_df_us.year.mode()[0], inplace=True)


In [1870]:
# See if in column year, are any irregularities
shark_df_us['year'].unique()

array([2023., 2022., 2021., 2020., 2019., 2018., 2017., 2016., 2015.,
       2014., 2013., 2012., 2011., 2010., 2009., 2008., 2007., 2006.,
       2005., 2004., 2003., 2002., 2001., 2000., 1999., 1998., 1997.,
       1996., 1995., 1994., 1993., 1992., 1991., 1990., 1989., 1988.,
       1987., 1986., 1985., 1984., 1983., 1982., 1981., 1980., 1979.,
       1978., 1977., 1976., 1975., 1974., 1973., 1972., 1971., 1970.,
       1969., 1968., 1967., 1966., 1965., 1964., 1963., 1962., 1961.,
       1960., 1959., 1958., 1957., 1956., 1955., 1954., 1953., 1952.,
       1951., 1950., 1949., 1948., 1947., 1945., 1944., 1943., 1941.,
       1940., 1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932.,
       1931., 1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923.,
       1922., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1892.,

In [1871]:
# Change all strings to integers
shark_df_us['year'] = shark_df_us['year'].astype(int)

C:\Users\vonke\AppData\Local\Temp\ipykernel_15648\427528797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us['year'] = shark_df_us['year'].astype(int)


In [1872]:
# Filter DataFrame for years with less than 4 chracters
condition = shark_df_us['year'].astype(str).apply(len) < 4
filtered_df = shark_df_us[condition]
filtered_df

,date,year,type,country,state,location,activity,name,sex,age,injury,time,species,activity_grouped
6818,2009?,0,Unprovoked,USA,south carolina,Charleston,Swimming,Rick Donnis,M,NaN,Minor injury,NaN,NaN,swimming
6822,Before 2012,0,Unprovoked,USA,hawaii,Oahu,Diving,Ken O'Keefe,M,NaN,Minor laceration to hand,NaN,"Galapagos shark, 6'",swimming
6824,Before 1908,0,Unprovoked,USA,california,"Monterey, Montery County",Fishing for basking sharks,males,M,NaN,FATAL PROVOKED INCIDENTS,NaN,Basking shark,water_sport
6831,Beforer 1994,0,Unprovoked,USA,florida,"Lost Tree Village, Palm Beach County",Surfing,C.M,M,NaN,Legs bitten,NaN,NaN,water_sport
6840,Before 2006,0,Unprovoked,USA,florida,"Tampa Bay, Hillsborough County",Wade-fishing,Ed Snyder,M,NaN,"No injury, shark rammed his back",NaN,NaN,fishing
6847,No date,0,Unprovoked,USA,florida,"Key West, Monroe County",Kitesurfing,Paul Menta,M,NaN,Hand bitten,NaN,NaN,water_sport
6856,"No date, Before 1975",0,Provoked,USA,florida,"Riviera Beach, Palm Beach County",Skin diving. Grabbed shark's tail; shark turne...,Carl Bruster,M,19,"Ankle punctured & lacerated, hands abraded PRO...",NaN,"Nurse shark, 2.1 m [7']",water_sport
6860,"No date, Before 1963",0,Unprovoked,USA,california,"LaJolla, San Diego County","Free diving, collecting sand dollars",Charles Fleming,M,NaN,Calf bitten,NaN,"Shovelnose guitarfish, adult male",swimming
6862,"No date, Before 1963",0,Invalid,USA,hawaii,"Portlock, Oahu",Diving,Val Valentine,M,NaN,A 4.3 m [14'] shark made threat display. No in...,NaN,Invalid,swimming
6863,"No date, Before 1902",0,Unprovoked,USA,florida,"Mosquito Inlet (Ponce Inlet), Volusia County",Canoeing,male,M,NaN,FATAL,NaN,NaN,water_sport


In [1873]:
# Get the unique value of these cells
result = shark_df_us['year'].unique()
smaller_than_4 = [value for value in result if len(str(value)) < 4]
print(smaller_than_4)

[0]


In [1874]:
# Replace these values with the Mode of the column year
for item in smaller_than_4:
    shark_df_us['year'] = shark_df_us['year'].replace(item, shark_df_us.year.mode()[0])

C:\Users\vonke\AppData\Local\Temp\ipykernel_15648\3655955125.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us['year'] = shark_df_us['year'].replace(item, shark_df_us.year.mode()[0])


In [1875]:
# Check again for irregularities
shark_df_us['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002,
       2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991,
       1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980,
       1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969,
       1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958,
       1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947,
       1945, 1944, 1943, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934,
       1933, 1932, 1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923,
       1922, 1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911,
       1910, 1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900,
       1899, 1898, 1897, 1896, 1895, 1894, 1892, 1891, 1890, 1889, 1887,
       1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879, 1878, 1876, 1874,
       1873, 1872, 1871, 1870, 1866, 1865, 1864, 18

# 2. Clean column Date

In [1876]:
# Check how many empty cells are in column date
shark_df_us['date'].isna().sum()

0

In [1877]:
# format
shark_df_us["date"] = shark_df_us["date"].str.replace('  ', '-').str.replace(' ', '-').str.replace('--', '-').str.replace('Reported', '').str.replace('`', '').str.replace('.', '').str.strip()
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: '-'.join(filter(lambda x: x and len(x) <= 10, date_list)))
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: '-'.join(x.split('-')) if pd.notna(x) and len(x.split('-')) > 1 else x)
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: date_list[:1] + [date_list[1][:3]] + date_list[2:] if len(date_list) == 3 else date_list).apply(lambda date_list: '-'.join(date_list) if date_list else '')
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: ['01', 'Jan'] + date_list if len(date_list) == 1 and len(date_list[0]) == 4 else date_list).apply(lambda date_list: '-'.join(date_list) if date_list else '')
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: ['01'] + date_list if len(date_list) == 2 else date_list).apply(lambda date_list: '-'.join(date_list) if date_list else '')
# shark_df_us["date"] = pd.to_datetime(shark_df_us["date"])

C:\Users\vonke\AppData\Local\Temp\ipykernel_15648\1182924623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us["date"] = shark_df_us["date"].str.replace('  ', '-').str.replace(' ', '-').str.replace('--', '-').str.replace('Reported', '').str.replace('`', '').str.replace('.', '').str.strip()
C:\Users\vonke\AppData\Local\Temp\ipykernel_15648\1182924623.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else [])

In [1878]:
# while True:
#     try:
#         shark_df_us["date"] = pd.to_datetime(shark_df_us["date"]) 
#         break
#     except:
#         shark_df_us["date"] = pd.to_datetime(None)
#         break

In [1879]:
# shark_df_us["date"] = pd.to_datetime(shark_df_us.date,format='%d-%m-%y')

In [1880]:
# shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: ['01', 'Jan'] + date_list if len(date_list) == 1 and len(date_list[0]) == 4 else date_list).apply(lambda date_list: '-'.join(date_list) if date_list else '')

In [1881]:
for format in date_formats:
    try:
        shark_df_us["date"] = pd.to_datetime(shark_df_us["date"])
        break  # Break out of the loop if a format is successfully applied
    except ValueError:
        print(f"Unable to parse dates with format {format}")
        print("Problematic dates:")
        print(shark_df_us.loc[pd.to_datetime(shark_df_us["date"], errors='coerce').isnull(), "date"])
        # shark_df_us["date"] = ''  # Set an empty string if the format is not successfully applied

# Now, shark_df_us["date"] should contain the parsed dates with the correct format, or empty strings for unsuccessful cases.

In [1882]:
# import pandas as pd

# date_formats = ['%d-%m-%y', '%d-%b-%Y', '%d-%B-%Y', '%d-%b%Y']

# for date_format in date_formats:
#     try:
#         shark_df_us["date"] = pd.to_datetime(shark_df_us["date"], format=date_format)
#         break  # Break out of the loop if successful
#     except ValueError:
#         pass  # Continue to the next format if unsuccessful
# else:
#     print(f"Unable to parse dates with formats {', '.join(date_formats)}")
#     print("Problematic dates:")
#     problematic_dates = shark_df_us.loc[pd.to_datetime(shark_df_us["date"], errors='coerce').isnull(), "date"]
#     print(problematic_dates)
#     # shark_df_us["date"] = ''  # Set an empty string if the formats are not successfully applied


In [1883]:
shark_df_us["date"].unique()

array(['30-Dec-2023', '05-Nov-2023', '25-Oct-2023', ..., '01-Before-1921',
       'Before-17-Jul-1916', '01-1900-1905'], dtype=object)

In [1884]:
shark_df_us['date']

2              30-Dec-2023
16             05-Nov-2023
21             25-Oct-2023
25             15-Oct-2023
26             13-Oct-2023
               ...        
6879        01-Before-1958
6913        01-Before-1921
6918    Before-17-Jul-1916
6922         01-Circa-1862
6929          01-1900-1905
Name: date, Length: 2476, dtype: object

In [1885]:
# Filter DataFrame  with less than 4 chracters
condition = shark_df_us['date']
filtered_df = shark_df_us[condition]
filtered_df

KeyError: "None of [Index(['30-Dec-2023', '05-Nov-2023', '25-Oct-2023', '15-Oct-2023',\n       '13-Oct-2023', '02-Oct-2023', '01-Oct-2023', '18-Sep-2023',\n       '12-Sep-2023', '11-Sep-2023',\n       ...\n       'No-date,-Before-1975', 'No-date,-Before-1963', 'No-date,-Before-1963',\n       'No-date,-Before-1902', 'Before-19-Jun-1959', '01-Before-1958',\n       '01-Before-1921', 'Before-17-Jul-1916', '01-Circa-1862',\n       '01-1900-1905'],\n      dtype='object', length=2476)] are in the [columns]"

In [ ]:
shark_df_us['date'].unique()

In [ ]:
text = '11-Dec-2021`'
print(text.strip())